https://www.kaggle.com/datasets/rikinzala/s-and-p-500-esg-and-stocks-data-2023-24

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rikinzala/s-and-p-500-esg-and-stocks-data-2023-24")

print("Path to dataset files:", path)



100%|██████████| 706k/706k [00:00<00:00, 1.57MB/s]

Extracting files...
Path to dataset files: C:\Users\stanc\.cache\kagglehub\datasets\rikinzala\s-and-p-500-esg-and-stocks-data-2023-24\versions\1


Acum am 2 csv uri: scorurile esg(dimensiunea etica) si preturile actiunilor(dimensiunea financiara)

In [37]:
import pandas as pd
import numpy as np

df_esg=pd.read_csv("../data/date_scoruri_esg.csv")
df_pret=pd.read_csv("../data/date_pret_actiuni.csv")

print("Tabel ESG încărcat. Rânduri:", len(df_esg))
print("Tabel Prețuri încărcat. Rânduri:", len(df_pret))





Tabel ESG încărcat. Rânduri: 426
Tabel Prețuri încărcat. Rânduri: 418


Calcularea Randamentelor si Unirea Datelor
-calculez cu cat a crescut sau a scazut fiecare actiune intre prima si ultima zi a setului de date 

In [38]:
preturi=df_pret.drop(columns=['Date']) #indepartez datele

randamente=(preturi.iloc[-1]/preturi.iloc[0])-1 #calculez randamentele

df_randamente=randamente.reset_index()
df_randamente.columns=['Symbol', 'Stock_return'] #creez un dataframe cu randamente

df_final=pd.merge(df_esg,df_randamente,on="Symbol",how="inner") #am creat dataframe ul cu dimensiunea etica si financiara

print(f"Date unite cu succes! Avem {len(df_final)} companii pentru analiză.")
df_final.head()


Date unite cu succes! Avem 426 companii pentru analiză.


,Symbol,Full Name,GICS Sector,GICS Sub-Industry,environmentScore,socialScore,governanceScore,totalEsg,highestControversy,percentile,ratingYear,ratingMonth,marketCap,beta,overallRisk,Stock_return
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,1.12,6.42,6.10,13.64,2.0,7.98,2023.0,9.0,39751831552,1.054,8,-0.047454
1,AAL,American Airlines Group,Industrials,Passenger Airlines,9.94,11.65,4.76,26.35,2.0,54.48,2023.0,9.0,7335383552,1.433,10,-0.166405
2,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",0.46,7.39,9.37,17.22,3.0,17.82,2023.0,9.0,3296096681984,1.240,1,0.830975
3,ABBV,AbbVie,Health Care,Biotechnology,2.38,17.19,10.36,29.93,3.0,68.62,2023.0,9.0,341698445312,0.619,6,0.208954
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,2.27,14.24,8.33,24.83,3.0,48.27,2023.0,9.0,205534380032,0.722,7,0.033674


Curatarea datelor nule

In [39]:
variabile_cheie=['governanceScore','Stock_return','marketCap','environmentScore']
print("Situația valorilor lipsă:")
print(df_final[variabile_cheie].isnull().sum())


Situația valorilor lipsă:
governanceScore     0
Stock_return        0
marketCap           0
environmentScore    0
dtype: int64


(Aparent nu exista date lipsa)

Standardizarea (z-score)
-aducem variabilele la aceeasi scara
z= x-xMediu/deviatiaStandard

In [40]:
media=df_final[variabile_cheie].mean()
deviatia=df_final[variabile_cheie].std()

df_standardizat=(df_final[variabile_cheie]-media)/deviatia

# 4. Verificăm rezultatul
# Acum toate coloanele ar trebui să aibă media ~0 și deviația standard 1
print("Noile medii (aproape de 0):")
print(df_standardizat.mean().round(4))
print("\nNoile deviații standard (egale cu 1):")
print(df_standardizat.std())

Noile medii (aproape de 0):
governanceScore    -0.0
Stock_return        0.0
marketCap           0.0
environmentScore   -0.0
dtype: float64

Noile deviații standard (egale cu 1):
governanceScore     1.0
Stock_return        1.0
marketCap           1.0
environmentScore    1.0
dtype: float64


DataFrame nou care combină identitatea companiilor cu scorurile standardizate

In [42]:
df_export=pd.concat(
    [
        df_final[['Symbol',"Full Name","GICS Sector"]].reset_index(drop=True),
        df_standardizat.reset_index(drop=True)
    ],axis=1)
df_export.to_csv("../data/processed/date_standardizate.csv",index=False)

print("Succes! Fișierul 'date_standardizate.csv' a fost creat.")

Succes! Fișierul 'date_standardizate.csv' a fost creat.
